In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import ndimage as ndimage, spatial, optimize, interpolate as interp
from skimage.filters import gaussian, threshold_otsu
from topostats.utils import update_config
import cv2
from scipy.ndimage import gaussian_filter
from skimage.filters import threshold_otsu
from topostats.tracing import dnatracing
from skimage.filters import gaussian
from topostats.io import read_yaml

%matplotlib inline

In [ ]:
def load_png(img_path):
    """Loads in a png image and converts this into a binary mask using standard thresholding"""
    image = Image.open(img_path)
    image = image.convert("L")  # Convert image to grayscale
    image = np.array(image)
    return image


def dilate_image(image):
    """Dilates the grayscale image using a specified kernel size"""

    # Convert nm to pixels based on the assumption that 16 pixels correspond to 5nm
    desired_width_pixels = 16

    # Create a structuring element with the desired width in pixels
    structuring_element = np.ones((desired_width_pixels, desired_width_pixels), dtype=np.uint8)

    # Perform dilation
    dilated_image = cv2.dilate(np.array(image), structuring_element)
    dilated_image = gaussian_filter(dilated_image, sigma=10)
    threshold_value = threshold_otsu(dilated_image)

    # Binarize the image using Otsu's threshold
    masked_image = (dilated_image > threshold_value).astype(np.uint8)
    return dilated_image, masked_image

In [ ]:
image = load_png("/Users/laurawiggins/Desktop/Topo data/Topology_simulations/nicked/dna-struct-AFM-run2-118.png")
dilated_image, masked_image = dilate_image(image)

plt.subplot(1, 2, 1)
plt.imshow(dilated_image, "afmhot")
plt.title("Image")

plt.subplot(1, 2, 2)
plt.imshow(masked_image, "viridis")
plt.title("Mask")

# Show the plot
plt.show()

In [ ]:
config = read_yaml("/Users/laurawiggins/Desktop/Topology simulations/TopoStats/config.yaml")
loading_config = config["loading"]
filter_config = config["filter"]
filter_config.pop("run")
grain_config = config["grains"]
grain_config.pop("run")
grainstats_config = config["grainstats"]
grainstats_config.pop("run")
dnatracing_config = config["dnatracing"]
dnatracing_config.pop("run")
plotting_config = config["plotting"]
plotting_config.pop("run")
plotting_config["image_set"] = "all"

results, node_dict, node_image_dict, images, splined_traces = dnatracing.trace_grain(
    dilated_image,
    masked_image,
    0.3,
    dnatracing_config["skeletonisation_params"],
    dnatracing_config["pruning_params"],
    "FILENAME",
    10,
    1,
)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(16, 4), dpi=200)

# Plot the additional image
axs[0].imshow(dilated_image, "afmhot")
axs[0].set_title("Pseudo-AFM")

# Plot masked_image
axs[1].imshow(masked_image, "viridis")
axs[1].set_title("Mask")

# Plot images["visual"]
axs[2].imshow(images["visual"], "viridis")
axs[2].set_title("Trace")

# Plot splined traces
axs[3].set_title("Spline")
axs[3].set_xlabel("X")
axs[3].set_ylabel("Y-")
for trace in splined_traces:
    x_values = trace[:, 0]
    y_values = trace[:, 1]
    axs[3].plot(y_values, -x_values)

# Adjust aspect ratio of the splined plot to match others
axs[3].set_aspect("equal")

plt.tight_layout()
plt.show()

In [ ]:
import os
import matplotlib.pyplot as plt


def process_folder(folder_path):
    node_dicts_list = []

    # Get the list of files in the folder and sort them
    files = sorted([filename for filename in os.listdir(folder_path) if filename.endswith(".png")])

    # Process every 20th file
    for i in range(0, len(files), 20):
        filename = files[i]
        filepath = os.path.join(folder_path, filename)
        image = load_png(filepath)  # Load the PNG image (replace load_png with your actual function)
        dilated_image, masked_image = dilate_image(image)  # Dilate and mask the image
        results, node_dict, node_image_dict, images, splined_traces = dnatracing.trace_grain(
            dilated_image,
            masked_image,
            0.3,
            dnatracing_config["skeletonisation_params"],
            dnatracing_config["pruning_params"],
            filename,
            10,
            1,
        )  # Perform tracing

        # Create a new dictionary to hold the molecule information including the filename
        updated_node_dict = {"filename": filename}
        updated_node_dict.update(node_dict)
        node_dicts_list.append(updated_node_dict)
        fig, axs = plt.subplots(1, 4, figsize=(16, 4), dpi=100)  # Create subplots

        # Plot the additional image
        axs[0].imshow(dilated_image, "afmhot")
        axs[0].set_title("Pseudo-AFM")

        # Plot masked_image
        axs[1].imshow(masked_image, "viridis")
        axs[1].set_title("Mask")

        # Plot images["visual"]
        axs[2].imshow(images["node_img"], "viridis")
        axs[2].set_title("Trace")

        # Plot splined traces
        axs[3].set_title("Spline")
        axs[3].set_xlabel("X")
        axs[3].set_ylabel("Y")
        for trace in splined_traces:
            x_values = trace[:, 0]
            y_values = trace[:, 1]
            axs[3].plot(y_values, -x_values)

        # Adjust aspect ratio of the splined plot to match others
        axs[3].set_aspect("equal")

        plt.tight_layout()
        plt.show()

    return node_dicts_list


# Example usage:
folder_path = "/Users/laurawiggins/Desktop/Topo data/Topology_simulations/nicked"
folder_node_dict = process_folder(folder_path)

In [ ]:
def calculate_classes(node_dicts):
    class_open = 0
    class_taut = 0
    class_clus = 0
    class_bowtie = 0
    class_undefined = 0

    for node_dict in node_dicts:  # Iterate over node_dicts
        filename = node_dict["filename"]  # Get the filename
        # Calculate the number of nodes and their branches
        node_count = len(node_dict) - 1  # Exclude the 'filename' key
        branches_counts = [len(node_data["branch_stats"]) for key, node_data in node_dict.items() if key != "filename"]

        # Check classification criteria
        if node_count == 5 and all(count == 2 for count in branches_counts):
            class_open += 1
        elif node_count == 4 and all(count == 2 for count in branches_counts):
            class_open += 1
        elif node_count == 3 and all(count == 2 for count in branches_counts):
            class_open += 1
        elif node_count == 2 and all(count == 2 for count in branches_counts):
            class_taut += 1
        elif node_count == 1 and all(count == 3 for count in branches_counts):
            class_clus += 1
        elif node_count == 1 and all(count == 2 for count in branches_counts):
            class_bowtie += 1
        else:
            class_undefined += 1
            print(f"Undefined class for filename: {filename}, node count: {node_count}")

    return {
        "class_open": class_open,
        "class_taut": class_taut,
        "class_clus": class_clus,
        "class_bowtie": class_bowtie,
        "class_undefined": class_undefined,
    }


# Example usage:
# Assuming node_dicts is a list containing multiple node_dicts
class_counts = calculate_classes(folder_node_dict)
print(class_counts)

In [ ]:
print(folder_node_dict)